In [1]:
from src.code_gen.code_generator import CCodeGenerator

generator = CCodeGenerator()

In [3]:
code = generator('''  
    let a = "5" in print(parse(a @ "22" @ a));
    '''
, False)

print(code)


Object* letInNode0();

Object* letInNode0() {
   Object* v0 = copyObject(createString("5"));
   return function_print(copyObject(function_parse(copyObject(stringConcat(stringConcat(v0, createString("22")), v0)))));
}


int main() {
   srand(time(NULL));

   letInNode0();
   return 0; 
}
